In [ ]:
# train.drop(['ADDRESS'],axis=1,inplace=True)
# test.drop(['ADDRESS'],axis=1,inplace=True)

# 데이터 불러오기

- ID - 각 집의 고유한 번호
- ADDRESS - 집의 주소
- SUBURB - 동네 이름
- PRICE - 가격
- BEDROOMS - 침실의 갯수
- BATHROOMS - 욕실의 갯수
- GARAGE - 차고의 수
- LAND_AREA - 토지 면적
- FLOOR_AREA - 건물 면적
- BUILD_YEAR - 건축년도
- CBD_DIST - Central business district까지의 거리
- NEAREST_STN - 근처 역 정보
- NEAREST_STN_DIST - 근처 역까지 거리
- DATE_SOLD - 판매된 날짜
- POSTCODE - 우편번호
- LATITUDE - 위도
- LONGITUDE - 경도
- NEAREST_SCH - 근교의 학교
- NEAREST_SCH_DIST - 근교의 학교까지의 거리
- NEAREST_SCH_RANK - 근교의 학교까지의 랭킹

In [ ]:
# pd.options.display.float_format = '{:.5f}'.format

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

train = pd.read_csv("./data/kaggle_r/train.csv",index_col='ID')
test = pd.read_csv("./data/kaggle_r/test.csv",index_col='ID')
submission = pd.read_csv("./data/kaggle_r/sample_submission.csv",index_col='ID')


In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

### 데이터불러오기 끝

# 데이터 이해

In [ ]:
train.columns

In [ ]:
test.columns

- PRICE 는 라벨데이터

In [ ]:
set(train.columns)-set(test.columns)

In [ ]:
train.info()

In [ ]:
train.select_dtypes(object).columns

In [ ]:
test.info()

## 데이터 접근
- 'ADDRESS', 'SUBURB', 'NEAREST_STN', 'DATE_SOLD', 'NEAREST_SCH'  범주형


- train 결측치 : NEAREST_SCH_RANK, BUILD_YEAR, GARAGE
- test 결측치 : NEAREST_SCH_RANK, BUILD_YEAR, GARAGE
    - NEAREST_SCH_RANK : 결측치 6000개 30%의 결측치
    - BUILD_YEAR : 결측치 1728개 10%정도의 결측치

- 상관관계
    - LAND_AREA:0.04678
    - NEAREST_STN_DIST:-0.09937
    - LATITUDE:0.04672
    - NEAREST_SCH_DIST:-0.02403

In [ ]:
train.info()

In [ ]:
train.corr()

## PRICE 가격 라벨데이터

In [ ]:
train.PRICE.info()

In [ ]:
train[['PRICE']].describe()

In [ ]:
train.PRICE.skew()

In [ ]:
sns.kdeplot(data=train,x='PRICE')

In [ ]:
sns.histplot(data=train,x='PRICE',bins=500)

## ADDRESS 집의 주소

In [ ]:
train[['ADDRESS']].describe()

In [ ]:
train['ADDRESS'].value_counts()

### ADDRESS 끝
- 총 data 가 18510 개 이지만 unique 값이 18491이므로 drop

In [ ]:
# # ADDRESS 드랍
# train = train.drop(['ADDRESS'], axis=1)
# test = test.drop(['ADDRESS'], axis=1)

## SUBURB 동네 이름
- 클래스는 318개 이니만 분포는 고른것으로 추정

In [ ]:
train[['SUBURB']].describe()

In [ ]:
train.SUBURB.unique()

In [ ]:
a = train['SUBURB'].values
a= set(a)
len(a)
a= list(a)
len(a)

In [ ]:
# for i in a:
    
#     sns.kdeplot(data=train[train['SUBURB']==i],x='PRICE',hue='SUBURB')
# #     plt.show()
    

In [ ]:
b = train['SUBURB'].value_counts()

In [ ]:
pd.DataFrame(b).head(10)

In [ ]:
# plt.figure(figsize=(15,5))
sns.histplot(data=train[train['SUBURB']=='Bertram'],x='PRICE',hue='SUBURB')

### SUBURB 끝
- 위도와 경도와 겹칠것으로 판단
- 드랍

In [ ]:
# # SUBURB 드랍
# train = train.drop(['SUBURB'], axis=1)
# test = test.drop(['SUBURB'], axis=1)

## BEDROOMS 침실 개수
- 방이 많을 수록 비싸지만 방개수는 3,4개가 많음

In [ ]:
train[['BEDROOMS']].describe()

In [ ]:
train['BEDROOMS'].info()

In [ ]:
train['BEDROOMS'].unique()

In [ ]:
train['BEDROOMS'].value_counts()

In [ ]:
sns.lineplot(y=train.PRICE,x=train.BEDROOMS)

In [ ]:
sns.countplot(data=train,x=train.BEDROOMS)

In [ ]:
sns.kdeplot(data=train,x="BEDROOMS")

In [ ]:
train.BEDROOMS.skew()

### BEDROOMS 끝
- 분포도 고르고 편향도 거의 없어서 나두기

## GARAGE 차고의수 결측치있음

In [ ]:
train.GARAGE.info()

In [ ]:
train.GARAGE = train.GARAGE.fillna(0)

In [ ]:
train.GARAGE.astype('int64')

In [ ]:
train[['GARAGE']].describe()
# 평균,중간값 비슷하지만 최대값과 차이남

In [ ]:
train.GARAGE.value_counts()

In [ ]:
train[train['GARAGE']==99]

In [ ]:
train['GARAGE'].replace(99,2,inplace=True)

In [ ]:
train.GARAGE.value_counts()

In [ ]:
train.GARAGE.info()

In [ ]:
sns.distplot(x=train["GARAGE"])

In [ ]:
train.GARAGE.skew()

In [ ]:
plt.xticks(rotation = 90 )
sns.countplot(data=train,x=train.GARAGE)

### GARAGE 끝
- 차고가 없는것이 잇을수잇으므로 결측치 0으로
- 99는 이상치로 판단하며, 개수가 1개이므로 가장많은 2에 편입

In [ ]:
# # GARAGE 결측치 처리
# train['GARAGE'] = train['GARAGE'].fillna(0)
# test['GARAGE'] = test['GARAGE'].fillna(0)

# train['GARAGE'].replace(99,2,inplace=True) # test에는 99없음

## LAND_AREA  토지 면적

In [ ]:
train[['LAND_AREA']].describe()

In [ ]:
train[['LAND_AREA']].value_counts()

In [ ]:
LAND_uni = test.LAND_AREA.unique()
LAND_uni = list(a)
LAND_uni.sort(reverse=True)
LAND_uni

In [ ]:
LAND_uni = train.LAND_AREA.unique()
LAND_uni = list(a)
LAND_uni.sort(reverse=True)
LAND_uni

In [ ]:
train[train['LAND_AREA']==999999]

In [ ]:
train[train['LAND_AREA']==983690]

In [ ]:
train[train['PRICE']==545000][['LAND_AREA']].value_counts()

In [ ]:
train[(365000<=train['PRICE'])&(train['PRICE']<=385000)][['LAND_AREA']].value_counts()

In [ ]:
# sns.lineplot(data=train,x="LAND_AREA",y='PRICE')

### LAND_AREA 끝
- 999999, 983690 두개 이상치로 추정 비슷한 가격대의 값으로 편입
- 999999,983690->692


- 스케일링,정규화 필요


- test,train 값이 거의 비슷하던데 확인필요?

In [ ]:
# # LAND_AREA 이상치 처리
# train['LAND_AREA'].replace(999999,692,inplace=True)
# train['LAND_AREA'].replace(983690,692,inplace=True)

# test['LAND_AREA'].replace(999999,692,inplace=True)
# test['LAND_AREA'].replace(983690,692,inplace=True)

# train['LAND_AREA'] = np.log(train['LAND_AREA'] + 1)
# test['LAND_AREA'] = np.log(test['LAND_AREA'] + 1)

In [ ]:
train.LAND_AREA.skew()

In [ ]:
test.LAND_AREA.skew()

## FLOOR_AREA 건물면적

In [ ]:
train[['FLOOR_AREA']].describe()

In [ ]:
train.FLOOR_AREA.value_counts()

In [ ]:
sns.scatterplot(x=train.FLOOR_AREA,y=train.PRICE)

In [ ]:
sns.boxplot(data=train,x='FLOOR_AREA')

In [ ]:
train.FLOOR_AREA.skew()

### FLOOR_AREA 끝
- 이상치 처리가 필요해보이나 일단 방법을 모르므로 패스

## BUILD_YEAR 건축년도 결측치 있음

In [ ]:
train.BUILD_YEAR.info

In [ ]:
train[['BUILD_YEAR']].describe()

In [ ]:
train.BUILD_YEAR.value_counts()

In [ ]:
train.BUILD_YEAR.unique()

In [ ]:
train.BUILD_YEAR = train.BUILD_YEAR.fillna(9999)

In [ ]:
train.BUILD_YEAR.info()

In [ ]:
train.info()

In [ ]:
plt.figure(figsize=(20,5))
plt.xticks(rotation=90)
sns.countplot(data=train,x="BUILD_YEAR")

In [ ]:
train.groupby(['BUILD_YEAR']).mean()['PRICE']

In [ ]:
# sns.kdeplot(data=train,x='BUILD_YEAR',hue='PRICE')

### BUILD_YEAR 끝
- 결측치 제거 방법도 잘모르겠음 일단 드랍?

In [ ]:
# # BUILD_YEAR 드랍
# train = train.drop(['BUILD_YEAR'], axis=1)
# test = test.drop(['BUILD_YEAR'], axis=1)

## CBD_DIST Central business district까지의 거리

In [ ]:
train[['CBD_DIST']].describe()

In [ ]:
train.CBD_DIST.skew()

In [ ]:
sns.boxplot(data=train,x='CBD_DIST')

### CBD_DIST 끝
- 이상치가 존재하는지는 잘모르겠으며 처리가 필요한지 판단이 안서므로 일단 그냥 사용

## NEAREST_STN - 근처 역 정보

In [ ]:
train[['NEAREST_STN']].describe()

In [ ]:
train.NEAREST_STN.value_counts()

In [ ]:
sns.scatterplot(data=train,x='NEAREST_STN',y='SUBURB')

### NEAREST_STN 끝
- 모르겠음 드랍

In [ ]:
# # NEAREST_STN 드랍
# train = train.drop(['NEAREST_STN'], axis=1)
# test = test.drop(['NEAREST_STN'], axis=1)

## NEAREST_STN_DIST - 근처 역까지 거리

In [ ]:
train[['NEAREST_STN_DIST']].describe()

In [ ]:
train.NEAREST_STN_DIST.skew()

In [ ]:
sns.boxplot(data=train,x='NEAREST_STN_DIST')

### NEAREST_STN_DIST
- 모르겠음

In [ ]:
# # NEAREST_STN_DIST 드랍
# train = train.drop(['NEAREST_STN_DIST'], axis=1)
# test = test.drop(['NEAREST_STN_DIST'], axis=1)

## DATE_SOLD - 판매된 날짜

In [ ]:
train[['DATE_SOLD']].describe()

In [ ]:
def split_m(row):
    for i in range(train['DATE_SOLD'].size):
        row = row[i][:7]
        month, year = row.split('-')
        return int(year+month)

# train['DATE_SOLD'] = train[['DATE_SOLD']].apply(split_m,axis=1)
# test['DATE_SOLD'] = test[['DATE_SOLD']].apply(split_m,axis=1)

In [ ]:
# train['DATE_SOLD'] = train[['DATE_SOLD']].apply(split_m,axis=1)

In [ ]:
train['DATE_SOLD'].info()

In [ ]:
train[['DATE_SOLD']].describe()

In [ ]:
train.DATE_SOLD.value_counts()

#### DATE_SOLD 끝
- 쓸모없는 문자열 숫자로 바꿔 순서가있는 범주형으로 변경

In [ ]:
# # DATE_SOLD 문자열 -> 정수형
# def split_m(row):
#     for i in range(train['DATE_SOLD'].size):
#         row = row[i][:7]
#         month, year = row.split('-')
#         return int(year+month)

# train['DATE_SOLD'] = train[['DATE_SOLD']].apply(split_m,axis=1)
# test['DATE_SOLD'] = test[['DATE_SOLD']].apply(split_m,axis=1)

## POSTCODE - 우편번호

In [ ]:
train['POSTCODE']=train['POSTCODE'].astype('object')

In [ ]:
train[['POSTCODE']].describe()

In [ ]:
train.POSTCODE.value_counts()

### POSTCODE 끝
- 위도와 경도와 거의 겹침 드랍

In [ ]:
# # POSTCODE 드랍
# train = train.drop(['POSTCODE'], axis=1)
# test = test.drop(['POSTCODE'], axis=1)

## LATITUDE - 위도

In [ ]:
train[['LATITUDE']].describe()

In [ ]:
sns.histplot(data=train,x='LATITUDE',bins=20)

### LATITUDE 끝
- 상관관계도 낮은데 표준편차가 거의 0
- 분포를 높여서 사용하는것 고려

## LONGITUDE - 경도

In [ ]:
train[['LONGITUDE']].describe()

### LONGITUDE 끝
- 상관관계도 낮은데 표준편차가 거의 0
- 분포를 높여서 사용하는것 고려

## NEAREST_SCH - 근교의 학교

In [ ]:
train[['NEAREST_SCH']].describe()

In [ ]:
train.NEAREST_SCH.value_counts()

### NEAREST_SCH 끝
- 근교의 학교까지의 거리가 상관관계가 0과 가까운데 근교의 학교가 상관이 거의 없을것으로 판단
- 범주가 너무많아 분류도 힘듬
- 드랍

In [ ]:
# # NEAREST_SCH 드랍
# train = train.drop(['NEAREST_SCH'], axis=1)
# test = test.drop(['NEAREST_SCH'], axis=1)

## NEAREST_SCH_DIST - 근교의 학교까지의 거리

In [ ]:
train.corr()

#### NEAREST_SCH_DIST 끝
- corr -0.02403 드랍

In [ ]:
# # NEAREST_SCH_DIST 드랍
# train = train.drop(['NEAREST_SCH_DIST'], axis=1)
# test = test.drop(['NEAREST_SCH_DIST'], axis=1)

## NEAREST_SCH_RANK - 근교의 학교까지의 랭킹

In [ ]:
train.NEAREST_SCH_RANK.unique()

In [ ]:
sns.scatterplot(x=train.NEAREST_SCH_RANK,y=train.PRICE)

In [ ]:
train[['NEAREST_SCH_RANK']].describe()

#### NEAREST_SCH_RANK 끝
- 드랍 결측치 30퍼 회생불가

In [ ]:
# # NEAREST_SCH_RANK 드랍
# train = train.drop(['NEAREST_SCH_RANK'], axis=1)
# test = test.drop(['NEAREST_SCH_RANK'], axis=1)

# 전처리 완료 코드모음

In [ ]:
# ADDRESS 드랍
train = train.drop(['ADDRESS'], axis=1)
test = test.drop(['ADDRESS'], axis=1)
# SUBURB 드랍
train = train.drop(['SUBURB'], axis=1)
test = test.drop(['SUBURB'], axis=1)
# BUILD_YEAR 드랍
train = train.drop(['BUILD_YEAR'], axis=1)
test = test.drop(['BUILD_YEAR'], axis=1)
# NEAREST_STN 드랍
train = train.drop(['NEAREST_STN'], axis=1)
test = test.drop(['NEAREST_STN'], axis=1)
# NEAREST_STN_DIST 드랍
train = train.drop(['NEAREST_STN_DIST'], axis=1)
test = test.drop(['NEAREST_STN_DIST'], axis=1)
# POSTCODE 드랍
train = train.drop(['POSTCODE'], axis=1)
test = test.drop(['POSTCODE'], axis=1)
# NEAREST_SCH 드랍
train = train.drop(['NEAREST_SCH'], axis=1)
test = test.drop(['NEAREST_SCH'], axis=1)
# NEAREST_SCH_DIST 드랍
train = train.drop(['NEAREST_SCH_DIST'], axis=1)
test = test.drop(['NEAREST_SCH_DIST'], axis=1)
# NEAREST_SCH_RANK 드랍
train = train.drop(['NEAREST_SCH_RANK'], axis=1)
test = test.drop(['NEAREST_SCH_RANK'], axis=1)

# GARAGE 결측치 처리
train['GARAGE'] = train['GARAGE'].fillna(0)
test['GARAGE'] = test['GARAGE'].fillna(0)
train['GARAGE'].replace(99,2,inplace=True) # test에는 99없음


# LAND_AREA 이상치 처리
train['LAND_AREA'].replace(999999,692,inplace=True)
train['LAND_AREA'].replace(983690,692,inplace=True)
test['LAND_AREA'].replace(999999,692,inplace=True)
test['LAND_AREA'].replace(983690,692,inplace=True)
train['LAND_AREA'] = np.log(train['LAND_AREA'] + 1)
test['LAND_AREA'] = np.log(test['LAND_AREA'] + 1)

# DATE_SOLD 문자열 -> 정수형
def split_m(row):
    for i in range(train['DATE_SOLD'].size):
        row = row[i][:7]
        month, year = row.split('-')
        return int(year+month)
train['DATE_SOLD'] = train[['DATE_SOLD']].apply(split_m,axis=1)
test['DATE_SOLD'] = test[['DATE_SOLD']].apply(split_m,axis=1)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.shape, test.shape

# 특성 라벨 분리

In [ ]:
X_train = train.drop('PRICE',axis=1)
y_train = train['PRICE']
X_train.shape,y_train.shape

In [ ]:
test.shape

# 스케일링 전부 적용

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# 스케일러 생성 (평균과 표준편차)
std_scaler = StandardScaler()

In [ ]:
# 기준값 생성 (평균,표준편차)
# 항상 2차원으로 넣어야함(DataFrame 형태)
std_scaler.fit(X_train)

In [ ]:
X_train_scale = std_scaler.transform(X_train)
X_test_scale = std_scaler.transform(test)

# 모델

In [ ]:
import math
import time

def code_s():
    start = time.time()
    math.factorial(100000)
    end = time.time()
    print(f"{end - start:.5f} sec")

In [ ]:
xgb = XGBRegressor(random_state=128,
                   learning_rate=0.1,
                   n_jobs=-1
                  )

In [ ]:
param_grid = {
"n_estimators": range(100, 501, 100),
"max_depth": range(1,6,1)
}

# return_train_score : 훈련 폴드(cv)의 점수를 받을지 여부를 설정
# scoring : 분류 (accuracy, f1), 회귀 (neg_mean_squared_error, r2)
# n_jobs : 병렬처리 수 (CPU 코어 수가 충분하다면 설정, 디폴트 1) - 내부적으로 멀티프로세스 동작 -> 속도 증가
#grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True, scoring='f1_micro', n_jobs=2)
grid_search = GridSearchCV(xgb, param_grid, cv=5)

# grid_search.fit(X_train, y_train)

In [ ]:
# grid_search.best_score_

In [ ]:
# grid_search.best_params_

In [ ]:
xgb = XGBRegressor(random_state=128,
                   n_jobs=-1,
                   learning_rate=0.1,
                   n_estimators=500,
                   max_depth = 5
                  )

In [ ]:
cros = cross_val_score(xgb,X_train_scale,y_train,cv=5)
cros.mean()

code_s()

In [ ]:
cros.mean()

In [ ]:
xgb.fit(X_train_scale,y_train)
code_s()

In [ ]:
pre = xgb.predict(X_test_scale)

In [ ]:
submission["PRICE"] = pre
submission.to_csv('data/XGB_R_0723_R02.csv')